In [43]:
import sys,os
sys.path.append("..")
import django
django.setup()
import datetime
import pandas as pd
import requests
from io import StringIO
from crawlers.finlab.data_process_tools import last_month
pd.set_option("display.max_rows", None)

In [38]:
"""
單季
STEP: DATA
STOCK_ID: 3293
RPT_CAT: XX_M_QUAR
QRY_TIME: 20194

"""

"""
累季
STEP: DATA
STOCK_ID: 3293
RPT_CAT: XX_M_QUAR_ACC
QRY_TIME: 20194

"""

'\n累季\nSTEP: DATA\nSTOCK_ID: 3293\nRPT_CAT: XX_M_QUAR_ACC\nQRY_TIME: 20194\n\n'

In [103]:
class CrawlGoodinfoSingleFRTW:
# stock_id='2330'
    def season_to_date(sym):
        year=int(sym[:4])
        season=sym[-2:]
        if season == 'Q4':
            date=datetime.date(year + 1, 3, 31)
        elif season == 'Q3':
            date=datetime.date(year, 8, 14)
        elif season == 'Q2':
            date=datetime.date(year, 11, 14)    
        else :
            date=datetime.date(year, 5, 15)
        return date
    
    def crawl_report(self,stock_id):
        # 偽瀏覽器
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
            'content-type': 'application/x-www-form-urlencoded;',
            'origin': 'https://goodinfo.tw',
            'referer': 'https://goodinfo.tw/StockInfo/StockFinDetail.asp?'
        }

        url='https://goodinfo.tw/StockInfo/StockFinDetail.asp?STEP=DATA&STOCK_ID='+stock_id+'&RPT_CAT=XX_M_QUAR&QRY_TIME=20194'

        r = requests.get(url, headers=headers)
        r.encoding = 'utf-8'
        html_df = pd.read_html(StringIO(r.text))
        df=pd.DataFrame(html_df[1])
        df.columns=df.iloc[0].apply(lambda s:s.replace('\\xa0', ' '))
        df=df.iloc[1:].set_index('獲利能力')
        df=df.T
        df=df.apply(lambda s: pd.to_numeric(s, errors='coerce'))
        df=df.reset_index().dropna(how='all',axis=1)
        df=df.rename(columns={0:'date'})
        df['stock_id']=stock_id
        df['date']=df['date'].apply(lambda s:season_to_date(s))
        return df

df=CrawlGoodinfoSingleFRTW().crawl_report('2330')
df

獲利能力,date,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,稅後淨利率 (母公司),每股稅前盈餘 (元),每股稅後盈餘 (元),每股淨值 (元),股東權益報酬率 (當季),股東權益報酬率 (年預估),資產報酬率 (當季),資產報酬率 (年預估),營收季成長率,毛利季成長率,營業利益季成長率,稅前淨利季成長率,稅後淨利季成長率,稅後淨利季成長率 (母公司),每股稅後盈餘季成長率,營收年成長率,毛利年成長率,營業利益年成長率,稅前淨利年成長率,稅後淨利年成長率,稅後淨利年成長率 (母公司),每股稅後盈餘年成長率,現金 (%),應收帳款 (%),存貨 (%),速動資產 (%),流動資產 (%),基金與投資 (%),固定資產 (%),無形資產 (%),其他資產 (%),現金季成長率,應收帳款季成長率,存貨季成長率,流動資產季成長率,基金與投資季成長率,固定資產季成長率,無形資產季成長率,其他資產季成長率,資產總額季成長率,現金年成長率,應收帳款年成長率,存貨年成長率,流動資產年成長率,基金與投資年成長率,固定資產年成長率,無形資產年成長率,其他資產年成長率,資產總額年成長率,應付帳款 (%),流動負債 (%),長期負債 (%),其他負債 (%),負債總額 (%),普通股股本 (%),股東權益總額 (%),應付帳款季成長率,流動負債季成長率,長期負債季成長率,其他負債季成長率,負債總額季成長率,普通股股本季成長率,股東權益總額季成長率,應付帳款年成長率,流動負債年成長率,長期負債年成長率,其他負債年成長率,負債總額年成長率,普通股股本年成長率,股東權益總額年成長率,現金比,速動比,流動比,利息保障倍數,現金流量比 (當季),現金流量比 (年預估),營業成本率,營業費用率,應收帳款週轉率 (次/年),應收款項收現日數 (日),應付帳款週轉率 (次/年),應付款項付現日數 (日),存貨週轉率 (次/年),平均售貨日數 (日),固定資產週轉率 (次/年),總資產週轉率 (次/年),淨值週轉率 (次/年),應收帳款佔營收比率 (當季),應收帳款佔營收比率 (年預估),存貨佔營收比率 (當季),存貨佔營收比率 (年預估),每股營業現金流量 (元),每股投資現金流量 (元),每股融資現金流量 (元),每股淨現金流量 (元),每股自由現金流量 (元),負債對淨值比率,長期資金適合率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分 (100為滿分),stock_id
0,2020-03-31,50.18,39.16,40.59,36.59,36.58,4.97,4.48,62.53,7.23,28.93,5.28,21.11,8.26,14.19,15.16,14.64,14.81,14.81,14.87,9.48,15.26,15.98,15.93,16.07,16.05,16.06,20.11,6.17,3.66,31.94,36.32,1.33,60.47,0.91,0.17,0.66,-3.89,-14.17,-3.16,0.12,12.68,14.15,-2.78,6.12,-21.19,8.17,-19.62,-13.56,2.96,27.76,21.47,16.54,8.36,15.48,26.08,1.11,0.51,28.38,11.45,71.62,26.60,19.39,0.00,-1.89,17.55,0,2.18,202.40,73.47,-55.89,-23.19,55.76,0,-3.30,77.09,122.4,139.3,201.10,34.36,137.4,49.82,11.02,8.90,41.03,2.01,181.20,7.04,51.87,0.98,0.58,0.79,44.08,11.02,26.16,6.54,7.84,-6.63,-0.66,0.11,1.21,39.62,120.3,9.86,1.43,3.52,96,2330
1,2019-08-14,47.57,36.82,38.33,34.50,34.49,4.33,3.90,61.20,6.44,25.74,4.62,18.49,21.60,34.40,41.39,39.47,51.41,51.38,51.75,12.56,13.01,13.27,13.59,13.47,13.47,13.37,21.20,6.82,4.53,34.22,39.80,1.41,56.95,0.85,0.18,-30.36,23.63,-10.67,-15.91,2.79,4.70,3.39,-8.66,-4.69,-7.43,12.25,-8.21,-1.14,1.11,15.92,29.34,7.60,8.34,12.98,23.18,1.18,0.55,25.62,12.15,74.38,-31.75,-20.49,-28.90,-0.88,-20.18,0,2.13,133.50,53.84,-55.89,-23.34,38.75,0,0.74,91.44,147.6,171.7,134.20,28.65,114.6,52.43,10.76,8.91,40.97,1.80,202.70,6.00,60.86,0.99,0.54,0.75,49.64,12.41,32.99,8.25,5.47,-4.18,-8.84,-7.61,1.29,34.44,132.7,10.00,1.52,3.96,96,2330
2,2019-11-14,43.04,31.66,33.42,27.71,27.70,3.10,2.57,59.92,4.05,16.20,3.02,12.07,10.19,14.80,18.73,18.13,8.78,8.75,8.44,3.31,-6.99,-9.62,-8.04,-7.63,-7.64,-7.89,29.01,5.25,4.83,39.40,45.11,1.31,51.84,0.78,0.19,0.62,10.19,-0.41,1.90,-2.41,3.04,-2.63,-3.01,2.37,2.76,28.77,9.29,5.33,1.66,12.24,26.89,12.18,9.05,18.12,27.79,1.58,0.53,30.59,11.58,69.41,186.50,64.50,0.00,-10.77,54.29,0,-10.85,25.89,34.40,-57.67,-26.80,21.70,0,4.28,104.40,141.8,162.3,94.14,18.92,75.7,56.96,11.38,8.59,42.49,2.01,181.90,5.06,72.10,0.84,0.44,0.58,48.83,12.21,44.91,11.23,4.53,-4.42,-0.04,0.16,0.12,44.06,136.9,17.10,1.76,5.27,89,2330
3,2019-05-15,41.31,29.38,31.18,28.07,28.07,2.63,2.37,67.21,3.59,14.36,2.87,11.48,-24.52,-34.58,-40.01,-38.62,-38.62,-38.60,-38.60,-11.84,-27.63,-33.63,-31.78,-31.63,-31.62,-31.50,29.52,4.88,4.97,39.64,45.32,1.37,51.51,0.82,0.20,11.74,-17.39,5.28,4.17,2.50,5.10,5.72,35.31,4.66,11.75,-1.04,27.54,6.70,-12.62,6.76,31.44,22.27,6.75,6.47,17.29,1.61,0.61,20.29,11.85,79.71,22.14,11.08,-37.96,-11.48,7.58,0,3.94,21.46,10.53,-57.67,-20.39,0.31,0,8.52,170.70,229.2,262.1,76.84,40.36,161.4,58.69,11.93,7.41,49.24,3.99,91.56,4.85,75.33,0.80,0.41,0.51,48.83,12.21,49.69,12.42,5.89,-2.48,-0.87,2.62,3.42,25.46,157.9,9.97,1.79,5.74,89,2330
4,2019-03-31,47.67,36.97,38.33,34.51,34.50,4.29,3.86,64.67,6.15,24.59,4.93,19.71,11.30,11.96,12.47,12.32,12.24,12.25,12.21,4.40,-0.45,-1.63,-0.53,0.70,0.70,0.78,27.64,6.18,4.94,39.44,45.53,1.40,51.29,0.81,0.16,18.23,-0.26,-2.00,10.76,-1.67,2.24,21.54,-10.23,6.10,4.41,5.53,39.73,11.02,-29.50,0.89,19.94,-23.01,4.93,5.55,16.29,2.72,0.72,19.74,12.41,80.26,-2.26,5.88,0.00,-2.08,4.71,0,6.45,-6.09,-5.06,-38.02,-18.24,-12.04,0,10.16,169.70,242.1,279.5,127.80,55.61,

In [104]:
c=list(df.columns)#.remove('date').remove('stock_id')
[for i in c]
# c=c[1:-1]
# {i:c for i,c in zip(c,en['col'])}

['date',
 '營業毛利率',
 '營業利益率',
 '稅前淨利率',
 '稅後淨利率',
 '稅後淨利率\xa0(母公司)',
 '每股稅前盈餘\xa0(元)',
 '每股稅後盈餘\xa0(元)',
 '每股淨值\xa0(元)',
 '股東權益報酬率\xa0(當季)',
 '股東權益報酬率\xa0(年預估)',
 '資產報酬率\xa0(當季)',
 '資產報酬率\xa0(年預估)',
 '營收季成長率',
 '毛利季成長率',
 '營業利益季成長率',
 '稅前淨利季成長率',
 '稅後淨利季成長率',
 '稅後淨利季成長率\xa0(母公司)',
 '每股稅後盈餘季成長率',
 '營收年成長率',
 '毛利年成長率',
 '營業利益年成長率',
 '稅前淨利年成長率',
 '稅後淨利年成長率',
 '稅後淨利年成長率\xa0(母公司)',
 '每股稅後盈餘年成長率',
 '現金\xa0(%)',
 '應收帳款\xa0(%)',
 '存貨\xa0(%)',
 '速動資產\xa0(%)',
 '流動資產\xa0(%)',
 '基金與投資\xa0(%)',
 '固定資產\xa0(%)',
 '無形資產\xa0(%)',
 '其他資產\xa0(%)',
 '現金季成長率',
 '應收帳款季成長率',
 '存貨季成長率',
 '流動資產季成長率',
 '基金與投資季成長率',
 '固定資產季成長率',
 '無形資產季成長率',
 '其他資產季成長率',
 '資產總額季成長率',
 '現金年成長率',
 '應收帳款年成長率',
 '存貨年成長率',
 '流動資產年成長率',
 '基金與投資年成長率',
 '固定資產年成長率',
 '無形資產年成長率',
 '其他資產年成長率',
 '資產總額年成長率',
 '應付帳款\xa0(%)',
 '流動負債\xa0(%)',
 '長期負債\xa0(%)',
 '其他負債\xa0(%)',
 '負債總額\xa0(%)',
 '普通股股本\xa0(%)',
 '股東權益總額\xa0(%)',
 '應付帳款季成長率',
 '流動負債季成長率',
 '長期負債季成長率',
 '其他負債季成長率',
 '負債總額季成長率',
 '普通股股本季成長率',
 '股東權益總額季成長率',
 '應付帳款年成長率',
 '流動負債年成長率',
 '

In [86]:
en=pd.read_csv('/Users/benbilly3/Downloads/無標題的試算表 - 工作表1.csv')
en['col']

0                                 Operating gross margin
1                                  Operating profit rate
2                           Net interest rate before tax
3                            Net interest rate after tax
4                 Net profit after tax  (parent company)
5                   Earnings per share before tax (yuan)
6                    Earnings per share after tax (yuan)
7                             Net value per share (yuan)
8            Return on shareholders' equity  (quarterly)
9      Return on shareholders' equity  (annual estimate)
10                         Return on assets  (quarterly)
11                   Return on assets  (annual estimate)
12                         Quarterly revenue growth rate
13                              Gross profit growth rate
14                                 Quarterly growth rate
15           Quarterly net profit growth rate before tax
16            Quarterly net profit growth rate after tax
17     Quarterly net profit gro

In [66]:
pd.set_option("display.max_columns", None)
# dd=df.rename(columns={i:1 for i in c})
# dd
df

獲利能力,date,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,稅後淨利率 (母公司),每股稅前盈餘 (元),每股稅後盈餘 (元),每股淨值 (元),股東權益報酬率 (當季),股東權益報酬率 (年預估),資產報酬率 (當季),資產報酬率 (年預估),獲利季成長率,營收季成長率,毛利季成長率,營業利益季成長率,稅前淨利季成長率,稅後淨利季成長率,稅後淨利季成長率 (母公司),每股稅後盈餘季成長率,獲利年成長率,營收年成長率,毛利年成長率,營業利益年成長率,稅前淨利年成長率,稅後淨利年成長率,稅後淨利年成長率 (母公司),每股稅後盈餘年成長率,各項資產佔總資產比重,現金 (%),應收帳款 (%),存貨 (%),速動資產 (%),流動資產 (%),基金與投資 (%),固定資產 (%),無形資產 (%),其他資產 (%),資產季成長率,現金季成長率,應收帳款季成長率,存貨季成長率,流動資產季成長率,基金與投資季成長率,固定資產季成長率,無形資產季成長率,其他資產季成長率,資產總額季成長率,資產年成長率,現金年成長率,應收帳款年成長率,存貨年成長率,流動資產年成長率,基金與投資年成長率,固定資產年成長率,無形資產年成長率,其他資產年成長率,資產總額年成長率,負債&股東權益佔總資產,應付帳款 (%),流動負債 (%),長期負債 (%),其他負債 (%),負債總額 (%),普通股股本 (%),股東權益總額 (%),負債&股東權益季增減率,應付帳款季成長率,流動負債季成長率,長期負債季成長率,其他負債季成長率,負債總額季成長率,普通股股本季成長率,股東權益總額季成長率,負債&股東權益年增減率,應付帳款年成長率,流動負債年成長率,長期負債年成長率,其他負債年成長率,負債總額年成長率,普通股股本年成長率,股東權益總額年成長率,償債能力,現金比,速動比,流動比,利息保障倍數,現金流量比 (當季),現金流量比 (年預估),經營能力,營業成本率,營業費用率,應收帳款週轉率 (次/年),應收款項收現日數 (日),應付帳款週轉率 (次/年),應付款項付現日數 (日),存貨週轉率 (次/年),平均售貨日數 (日),固定資產週轉率 (次/年),總資產週轉率 (次/年),淨值週轉率 (次/年),應收帳款佔營收比率 (當季),應收帳款佔營收比率 (年預估),存貨佔營收比率 (當季),存貨佔營收比率 (年預估),現金流量狀況,每股營業現金流量 (元),每股投資現金流量 (元),每股融資現金流量 (元),每股淨現金流量 (元),每股自由現金流量 (元),其他指標,負債對淨值比率,長期資金適合率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分 (100為滿分),stock_id
0,2020-03-31,50.18,39.16,40.59,36.59,36.58,4.97,4.48,62.53,7.23,28.93,5.28,21.11,NaN,8.26,14.19,15.16,14.64,14.81,14.81,14.87,NaN,9.48,15.26,15.98,15.93,16.07,16.05,16.06,NaN,20.11,6.17,3.66,31.94,36.32,1.33,60.47,0.91,0.17,NaN,0.66,-3.89,-14.17,-3.16,0.12,12.68,14.15,-2.78,6.12,NaN,-21.19,8.17,-19.62,-13.56,2.96,27.76,21.47,16.54,8.36,NaN,15.48,26.08,1.11,0.51,28.38,11.45,71.62,NaN,26.60,19.39,0.00,-1.89,17.55,0,2.18,NaN,202.40,73.47,-55.89,-23.19,55.76,0,-3.30,NaN,77.09,122.4,139.3,201.10,34.36,137.4,NaN,49.82,11.02,8.90,41.03,2.01,181.20,7.04,51.87,0.98,0.58,0.79,44.08,11.02,26.16,6.54,NaN,7.84,-6.63,-0.66,0.11,1.21,NaN,39.62,120.3,9.86,1.43,3.52,96,2330
1,2019-08-14,47.57,36.82,38.33,34.50,34.49,4.33,3.90,61.20,6.44,25.74,4.62,18.49,NaN,21.60,34.40,41.39,39.47,51.41,51.38,51.75,NaN,12.56,13.01,13.27,13.59,13.47,13.47,13.37,NaN,21.20,6.82,4.53,34.22,39.80,1.41,56.95,0.85,0.18,NaN,-30.36,23.63,-10.67,-15.91,2.79,4.70,3.39,-8.66,-4.69,NaN,-7.43,12.25,-8.21,-1.14,1.11,15.92,29.34,7.60,8.34,NaN,12.98,23.18,1.18,0.55,25.62,12.15,74.38,NaN,-31.75,-20.49,-28.90,-0.88,-20.18,0,2.13,NaN,133.50,53.84,-55.89,-23.34,38.75,0,0.74,NaN,91.44,147.6,171.7,134.20,28.65,114.6,NaN,52.43,10.76,8.91,40.97,1.80,202.70,6.00,60.86,0.99,0.54,0.75,49.64,12.41,32.99,8.25,NaN,5.47,-4.18,-8.84,-7.61,1.29,NaN,34.44,132.7,10.00,1.52,3.96,96,2330
2,2019-11-14,43.04,31.66,33.42,27.71,27.70,3.10,2.57,59.92,4.05,16.20,3.02,12.07,NaN,10.19,14.80,18.73,18.13,8.78,8.75,8.44,NaN,3.31,-6.99,-9.62,-8.04,-7.63,-7.64,-7.89,NaN,29.01,5.25,4.83,39.40,45.11,1.31,51.84,0.78,0.19,NaN,0.62,10.19,-0.41,1.90,-2.41,3.04,-2.63,-3.01,2.37,NaN,2.76,28.77,9.29,5.33,1.66,12.24,26.89,12.18,9.05,NaN,18.12,27.79,1.58,0.53,30.59,11.58,69.41,NaN,186.50,64.50,0.00,-10.77,54.29,0,-10.85,NaN,25.89,34.40,-57.67,-26.80,21.70,0,4.28,NaN,104.40,141.8,162.3,94.14,18.92,75.7,NaN,56.96,11.38,8.59,42.49,2.01,181.90,5.06,72.10,0.84,0.44,0.58,48.83,12.21,44.91,11.23,NaN,4.53,-4.42,-0.04,0.16,0.12,NaN,44.06,136.9,17.10,1.76,5.27,89,2330
3,2019-05-15,41.31,29.38,31.18,28.07,28.07,2.63,2.37,67.21,3.59,14.36,2.87,11.48,NaN,-24.52,-34.58,-40.01,-38.62,-38.62,-38.60,-38.60,NaN,-11.84,-27.63,-33.63,-31.78,-31.63,-31.62,-31.50,NaN,29.52,4.88,4.97,39.64,45.32,1.37,51.51,0.82,0.20,NaN,11.74,-17.39,5.28,4.17,2.50,5.10,5.72,35.31,4.66,NaN,11.75,-1.04,27.54,6.70,-12.62,6.76,31.44,22.27,6.75,NaN,6.47,17.29,1.61,0.61,20.29,11.85,79.71,NaN,22.14,11.08,-37.96,-11.48,7.58,0,3.94,NaN,21.46,10.53,-57.67,-20.39,0.31,0,8.52,NaN,170.70,229.2,262.1,76.84,40.36,161.4,NaN,58.69,11.93,7.41,49.24,3.99,91.56,4.85,75.33,0.80,0.41,0.51,48.83,12.21,49.69,12.42,NaN,5.89,-2.48,-0.87,2.62,3.42,NaN,25.46,157.9,9.97,1.79,5.74,89,2330
4,2019-03-31,47.67,36.97,38.33,34.51,34.50,4.29,3.86,64.67,6.15,24.59,4.93,19.71,NaN,11.30,11.96,12.47,12.32,12.24,12.25,12.21,NaN,4.40,-0.45,-1.63,-0.53,0.70,0.70,0.78,NaN

In [36]:
[i for i in df2.columns  if i not in df.columns]

[]

In [42]:
v= filter(str.isalpha, '每股融資現金流量 (元)')
address = ''.join(list(v))
address

'每股融資現金流量元'

In [93]:
'其他負債\xa0(%)'.replace('\xa0(%)','')

'其他負債'

In [25]:

def season_to_date(sym):
    year=int(sym[:4])
    season=sym[-2:]
    if season == 'Q4':
        date=datetime.date(year + 1, 3, 31)
    elif season == 'Q3':
        date=datetime.date(year, 8, 14)
    elif season == 'Q2':
        date=datetime.date(year, 11, 14)    
    else :
        date=datetime.date(year, 5, 15)
    return date
season_to_date('2019Q4')

datetime.date(2020, 3, 31)

In [16]:
df=df.apply(lambda s: pd.to_numeric(s, errors='coerce'))
df=df.reset_index().dropna(how='all',axis=1)
df

獲利能力,0,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,稅後淨利率 (母公司),每股稅前盈餘 (元),每股稅後盈餘 (元),每股淨值 (元),股東權益報酬率 (當季),...,每股營業現金流量 (元),每股投資現金流量 (元),每股融資現金流量 (元),每股淨現金流量 (元),每股自由現金流量 (元),負債對淨值比率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分 (100為滿分)
0,2019Q4,93.62,48.82,43.63,36.38,36.36,10.39,8.66,70.86,12.94,...,12.61,-3.74,0.09,8.96,8.87,32.85,16.60,-5.19,-11.90,91
1,2019Q3,92.39,46.89,47.28,39.91,39.88,9.98,8.42,62.61,14.39,...,9.07,5.67,-11.01,3.72,14.74,29.85,15.59,0.39,0.83,91
2,2019Q2,92.72,46.42,47.79,39.53,39.48,7.83,6.47,54.19,11.46,...,8.14,-19.61,-0.01,-11.47,-11.46,49.52,17.28,1.37,2.87,87
3,2019Q1,91.21,37.32,38.54,32.68,32.61,5.36,4.54,58.55,8.08,...,3.85,-2.45,0.43,1.84,1.40,22.34,15.20,1.22,3.16,96
4,2018Q4,87.36,33.29,35.00,30.11,29.97,4.39,3.76,53.54,7.28,...,5.48,-3.99,0.06,1.55,1.49,22.56,13.98,1.71,4.88,94
5,2018Q3,88.75,29.58,31.78,27.59,27.44,3.30,2.85,50.12,5.85,...,0.30,14.75,-11.01,4.03,15.05,18.82,13.19,2.20,6.91,85
6,2018Q2,86.78,31.51,39.83,32.58,32.44,4.19,3.41,47.54,6.63,...,4.71,-9.51,0.00,-4.80,-4.80,48.86,18.21,8.32,20.89,89
7,2018Q1,89.00,29.77,26.43,24.21,24.16,2.72,2.49,55.50,4.64,...,2.34,-3.47,0.19,-0.94,-1.13,20.40,8.40,-3.33,-12.61,85
8,2017Q4,87.23,33.48,31.45,27.84,27.86,3.56,3.15,52.05,6.23,...,3.77,-5.92,0.01,-2.13,-2.15,23.10,11.45,-2.04,-6.48,89
9,2017Q3,84.96,35.90,36.72,32.44,32.45,4.62,4.08,48.69,8.73,...,-0.14,10.34,-10.00,0.20,10.20,21.55,11.65,0.83,2.26,89


In [15]:
import pandas as pd
import requests

stock_id='2330'
headers2 = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded;',
    'origin': 'https://goodinfo.tw',
    'referer': 'https://goodinfo.tw/StockInfo/EquityDistributionClassHis.asp'
}

sess = requests.session()

res1 = requests.get('https://goodinfo.tw/StockInfo/EquityDistributionClassHis.asp?STOCK_ID='+stock_id+'&CHT_CAT=WEEK&STEP=DATA&DISPLAY_CAT=SHARES_NO',headers=headers2)

res1.encoding='utf-8'

df1 = pd.read_html(res1.text)
df1[2]

週別   統計日期   當月股價             各持股等級股東之持股張數(萬張)                   \
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
        週別   統計日期   當月股價  當月股價  當月股價 各持股等級股東之持股張數(萬張) 各持股等級股東之持股張數(萬張)   
        週別   統計日期     收盤 漲跌(元) 漲跌(%)            10張以下           10至50張   
0    20W16    NaN  286.5   7.0  2.50                -                -   
1    20W15  04/10  279.5   8.0  2.95            68.24            48.98   
2    20W14  04/01  271.5  -1.5 -0.55            69.11             49.5   
3    20W13  03/27  273.0   3.0  1.11            68.04            49.01   
4    20W12  03/20  270.0 -20.0 -6.90            69.77            50.01   
5    20W11  03/13  290.0 -25.0 -7.94            64.82            47.95   
6    20W10  03/06  315.0  -1.0 -0.32            60.18            46.55   
7    20W09  02/27  316.0  -9.0 -2.77            59.53            46.35   
8    20W08  02/21  325.0 -10.0 -2.99            58.15            46.03   
9    20W07  02/15  335.0   7.0  2.13            55.68            45.35   
10   20W06  02/07  328.0   8.0  2.50             56.5            45.76   
11   20W05  01/31  320.0 -13.0 -3.90             56.9            46.44   
12   20W04  01/22  333.0   0.0  0.00            55.67            45.88   
13   20W03  01/17  333.0  -6.5 -1.91            54.11            45.11   
14   20W02  01/10  339.5   0.0  0.00            53.09            44.58   
15   20W01  01/03  339.5   1.5  0.44             52.9            44.63   
16   19W52  12/27  338.0   9.0  2.74            52.65             44.5   
17   19W51  12/20  329.0 -10.0 -2.95            51.39            44.02   
18   19W50  12/13  339.0  26.0  8.31            48.71            43.09   
19   19W49  12/06  313.0   8.0  2.62            49.66            43.53   
20   19W48  11/29  305.0  -4.0 -1.29            49.11            43.25   
21   19W47  11/22  309.0   2.0  0.65            48.84            43.19   
22   19W46  11/15  307.0   1.5  0.49             49.2            43.35   
23   19W45  11/08  305.5   6.5  2.17            48.76             43.2   
24   19W44  11/01  299.0   5.5  1.87            48.49             43.2   
25   19W43  10/25  293.5   0.5  0.17            48.98             43.2   
26   19W42  10/18  293.0  11.0  3.90            48.12             42.7   
27   19W41  10/09  282.0   5.5  1.99            47.07            42.34   
28   19W40  10/05  276.5   4.5  1.65            47.31            42.34   
29   19W39  09/27  272.0   8.0  3.03            47.78            42.51   
..     ...    ...    ...   ...   ...              ...              ...   
126  17W45  11/10  240.5   1.5  0.63            44.15            41.05   
127  17W44  11/03  239.0   0.0  0.00            43.97            40.91   
128  17W43  10/27  239.0   1.5  0.63            43.86             40.9   
129  17W42  10/20  237.5   0.0  0.00             43.5            40.72   
130  17W41  10/13  237.5  13.0  5.79             43.3             40.6   
131  17W40  

In [ ]:
class CrawlFinancialReportTW:
    def __init__(self,date):
        self.date=date
        self.target_name = "台股月營收資訊"
        self.sub_market = ["sii", "otc", "rotc"]


    def crawl_main(self):
        url_date = last_month(self.date)
        data = []
        for i in self.sub_market:

            url = 'https://mops.twse.com.tw/nas/t21/' + i + '/t21sc03_' + str(url_date.year - 1911) + '_' + str(
                url_date.month) + '.html'

            # 偽瀏覽器
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)'
                              ' Chrome/39.0.2171.95 Safari/537.36'}

            # 下載該年月的網站，並用pandas轉換成 dataframe
            try:
                r = requests.get(url, headers=headers)
                r.encoding = 'big5'
                html_df = pd.read_html(StringIO(r.text))
                # 處理一下資料
                if html_df[0].shape[0] > 500:
                    df = html_df[0].copy()
                else:
                    df = pd.concat([df for df in html_df if (df.shape[1] <= 11) and (df.shape[1] > 5)])

                if 'levels' in dir(df.columns):
                    df.columns = df.columns.get_level_values(1)
                else:
                    df = df[list(range(0, 10))]
                    column_index = df.index[(df[0] == '公司代號')][0]
                    df.columns = df.iloc[column_index]

                df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
                df = df[~df['當月營收'].isnull()]
                df = df[df['公司代號'] != '合計']

                df['date'] = datetime.date(self.date.year, self.date.month, 10)

                df = df.rename(columns={'公司代號': 'stock_id'})
                df = df.set_index(['stock_id', 'date'])

                data.append(df)
            except Exception as e:
                print(e)
                print('**WARRN: Pandas cannot find any table in the HTML file')
                return None
        df = pd.concat(data)
        if '備註' not in df.columns:
            df['備註'] = None
        df.iloc[:, 1:-1] = df.iloc[:, 1:-1].apply(lambda s: pd.to_numeric(s, errors='coerce'))
        df = df[df['公司名稱'] != '總計']
        df = df.where(pd.notnull(df), None)
        df = df.rename(columns={'公司名稱': "stock_name", "當月營收": "this_month_rev",
                                '上月營收': "last_month_rev", "去年當月營收": "last_year_rev",
                                '上月比較增減(%)': "cp_last_month_rev", "去年同月增減(%)": "cp_last_year_rev",
                                '當月累計營收': "cm_this_month_rev", "去年累計營收": "cm_last_month_rev",
                                '前期比較增減(%)': "cp_cm_rev", "備註": "note",
                                })
        df = df.reset_index()

        return df

In [ ]:
C=CrawlMonthlyRevnueTW(datetime.datetime(2020,4,1))
C.crawl_main()

In [ ]:
#pycharm 測試
import sys,os
sys.path.append("..")
import django
django.setup()
from crawlers.models import *
# from crawlers.finlab.pioneers import *
# from crawlers.finlab.import_tools import *

In [ ]:
add_to_sql(CompanyBasicInfoTW, df)